In [29]:
# Install gdown if needed
# !pip install gdown

# import gdown
# gdown.download_folder('your_folder_name')


Retrieving folder contents


Processing file 1G4UfMB1ecY-OqVcHMfUMAih1B_ZFAf4x config.json
Processing file 1OeMWsh5cmQNdFVQU0oZL-Bu3BxXILPdn generation_config.json
Processing file 1A52l4M0oI2DOuLkArVPFy56YalQZbWnx merges.txt
Processing file 1w8HWz08pf7ImIWTpfCndh23xz1t8ZkzL model.safetensors
Processing file 1uOL3kUNc_MbFPWdomNK4p9mkUlNagwwR rng_state.pth
Processing file 1qMICI-uzpYoiy2G29feXgTs7U0o6H_Pe scaler.pt
Processing file 1tZ62BGy4ze2kEDrG7qfN30r89Op550a3 scheduler.pt
Processing file 1l0PXvn7WI0YMRjB4Wmh7NMDJ1DdT5vMH special_tokens_map.json
Processing file 1cZ9w8CNKeHDh1IvGN4OmaDDn28JXE2_f tokenizer_config.json
Processing file 1sskh5Ae6ZIKGJIMLhTAMiMRlTZpa32jy tokenizer.json
Processing file 1cANOyGF2cEyBu1-8OFTp8Dr_GBBV9wWZ trainer_state.json
Processing file 1iZ6emNFgnnGJzX_lLUKLqDZLhMwLbUIw training_args.bin
Processing file 1DzLvo0rIu_zomtcev6sfXSq0dC8FitsX vocab.json


Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1G4UfMB1ecY-OqVcHMfUMAih1B_ZFAf4x
To: /content/checkpoint-3600/config.json
100%|██████████| 1.59k/1.59k [00:00<00:00, 7.28MB/s]
Downloading...
From: https://drive.google.com/uc?id=1OeMWsh5cmQNdFVQU0oZL-Bu3BxXILPdn
To: /content/checkpoint-3600/generation_config.json
100%|██████████| 328/328 [00:00<00:00, 1.19MB/s]
Downloading...
From: https://drive.google.com/uc?id=1A52l4M0oI2DOuLkArVPFy56YalQZbWnx
To: /content/checkpoint-3600/merges.txt
100%|██████████| 456k/456k [00:00<00:00, 6.03MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1w8HWz08pf7ImIWTpfCndh23xz1t8ZkzL
From (redirected): https://drive.google.com/uc?id=1w8HWz08pf7ImIWTpfCndh23xz1t8ZkzL&confirm=t&uuid=65196c25-4817-42b3-8157-2e7f3dfdbf4c
To: /content/checkpoint-3600/model.safetensors
100%|██████████| 1.63G/1.63G [00:24<00:00, 65.9MB/s]
Downloading...
Fro

['/content/checkpoint-3600/config.json',
 '/content/checkpoint-3600/generation_config.json',
 '/content/checkpoint-3600/merges.txt',
 '/content/checkpoint-3600/model.safetensors',
 '/content/checkpoint-3600/rng_state.pth',
 '/content/checkpoint-3600/scaler.pt',
 '/content/checkpoint-3600/scheduler.pt',
 '/content/checkpoint-3600/special_tokens_map.json',
 '/content/checkpoint-3600/tokenizer_config.json',
 '/content/checkpoint-3600/tokenizer.json',
 '/content/checkpoint-3600/trainer_state.json',
 '/content/checkpoint-3600/training_args.bin',
 '/content/checkpoint-3600/vocab.json']

In [30]:
# Install dependencies if needed
# !pip install transformers datasets evaluate rouge-score
# !pip install sumy
# !pip install streamlit gradio
# !pip install nltk
# !pip install --upgrade transformers

In [31]:
from datasets import load_dataset
dataset = load_dataset("cnn_dailymail", name="3.0.0")
print(dataset)
print(dataset["train"][0])

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 11490
    })
})
{'article': 'LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won\'t cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. "I don\'t plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar," he told an Australian interviewer earlier this month. 

In [32]:
RANDOM_SEED=63
# Select smaller sets so training is achievable
small_train = dataset["train"].shuffle(seed=RANDOM_SEED).select(range(25000))
small_val = dataset["validation"].shuffle(seed=RANDOM_SEED).select(range(2000))

In [33]:
from transformers import pipeline

summarizer = pipeline("summarization",model="facebook/bart-large-cnn")

text=dataset["test"][0]["article"]
print("Original", text[:500],"...")
print("Reference: ", dataset["test"][0]["highlights"])

summary = summarizer(text, max_length=130, min_length=30, do_sample=False)
print("Generated:", summary[0]['summary_text']) #Sanity check


Device set to use cuda:0


Original (CNN)The Palestinian Authority officially became the 123rd member of the International Criminal Court on Wednesday, a step that gives the court jurisdiction over alleged crimes in Palestinian territories. The formal accession was marked with a ceremony at The Hague, in the Netherlands, where the court is based. The Palestinians signed the ICC's founding Rome Statute in January, when they also accepted its jurisdiction over alleged crimes committed "in the occupied Palestinian territory, includin ...
Reference:  Membership gives the ICC jurisdiction over alleged crimes committed in Palestinian territories since last June .
Israel and the United States opposed the move, which could open the door to war crimes investigations against Israelis .
Generated: The Palestinian Authority becomes the 123rd member of the International Criminal Court. The move gives the court jurisdiction over alleged crimes in Palestinian territories. Israel and the United States opposed the Palestinians' 

In [34]:
# If needed:
# !pip install sumy
import nltk
nltk.download('punkt_tab')

from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.text_rank import TextRankSummarizer

text = dataset["test"][0]["article"]

parser = PlaintextParser.from_string(text, Tokenizer("english"))
summarizer = TextRankSummarizer()

Extractive: "As Palestine formally becomes a State Party to the Rome Statute today, the world is also a step closer to ending a long era of impunity and injustice," he said, according to an ICC news release. Judge Kuniko Ozaki, a vice president of the ICC, said acceding to the treaty was just the first step for the Palestinians. "As the Rome Statute today enters into force for the State of Palestine, Palestine acquires all the rights as well as responsibilities that come with being a State Party to the Statute.


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [35]:
!pip install evaluate
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Trainer, TrainingArguments
import evaluate
import numpy as np

checkpoint = "facebook/bart-large-cnn"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

# Tokenization
def preprocess(batch):
    inputs = tokenizer(batch["article"],
                       max_length=512,
                       truncation=True,
                       padding=True)

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(batch["highlights"],
                           max_length=128,
                           truncation=True,
                           padding=True)
    inputs["labels"] = labels["input_ids"]
    return inputs

# Dataset.map - applying the tokenizer function to each element of the dataset
tokenized_train = small_train.map(
    preprocess,
    batched=True,
    remove_columns=["article", "highlights", "id"]  # Remove original columns
)
tokenized_val = small_val.map(
    preprocess,
    batched=True,
    remove_columns=["article", "highlights", "id"]  # Remove original columns
)



Map:   0%|          | 0/100 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:4006: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [37]:
#!pip install rouge_score
#!pip install evaluate
import os
import gc
import glob
import numpy as np
import torch
import evaluate
from transformers import (
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    DataCollatorForSeq2Seq,
)

# Cleaning GPU (working on Colab)
torch.cuda.empty_cache()
gc.collect()

# Might be useful for VRAM management
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

from google.colab import drive
drive.mount('/content/drive')

# Complicated metrics function with exceptions to prevent overflow which happened without them
rouge = evaluate.load("rouge")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred

    print(f"Predictions shape: {predictions.shape}")
    print(f"Predictions dtype: {predictions.dtype}")
    print(f"Predictions min/max: {predictions.min()}/{predictions.max()}")

    # Handle predictions - they might be logits, so take argmax
    if predictions.ndim == 3:  # If predictions are logits (batch_size, seq_len, vocab_size)
        print("Taking argmax of 3D predictions (logits)")
        predictions = np.argmax(predictions, axis=-1)

    print(f"After processing - Predictions min/max: {predictions.min()}/{predictions.max()}")

    # Clip predictions to valid token ID range
    max_token_id = tokenizer.vocab_size - 1 if hasattr(tokenizer, 'vocab_size') else 50000
    print(f"Tokenizer vocab size: {getattr(tokenizer, 'vocab_size', 'Unknown')}")
    print(f"Clipping to max_token_id: {max_token_id}")

    predictions = np.clip(predictions, 0, max_token_id)

    # Replace -100s in predictions with pad token id
    predictions = np.where(predictions != -100, predictions, tokenizer.pad_token_id)

    # Ensure predictions are int32 and within valid range
    predictions = predictions.astype(np.int32)

    try:
        # Decode predictions
        decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
        print(f"Successfully decoded {len(decoded_preds)} predictions")
    except (OverflowError, ValueError) as e:
        print(f"Error decoding predictions: {e}")
        # Fallback: create empty predictions
        decoded_preds = [""] * len(predictions)

    # Decode labels (replace -100 with pad token)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    labels = np.clip(labels, 0, max_token_id)  # Clip labels too for safety
    labels = labels.astype(np.int32)

    try:
        decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
        print(f"Successfully decoded {len(decoded_labels)} labels")
    except (OverflowError, ValueError) as e:
        print(f"Error decoding labels: {e}")
        # Fallback: create empty labels
        decoded_labels = [""] * len(labels)

    # Compute ROUGE only if we have valid decoded text
    if any(decoded_preds) and any(decoded_labels):
        try:
            result = rouge.compute(
                predictions=decoded_preds,
                references=decoded_labels,
                use_stemmer=True
            )

            print(f"ROUGE scores computed successfully")
            # Return selected ROUGE scores
            return {
                "rouge1": result["rouge1"],
                "rouge2": result["rouge2"],
                "rougeL": result["rougeL"],
            }
        except Exception as e:
            print(f"Error computing ROUGE: {e}")
            return {
                "rouge1": 0.0,
                "rouge2": 0.0,
                "rougeL": 0.0,
            }
    else:
        print("No valid decoded text found, returning zero scores")
        # Return zero scores if decoding failed
        return {
            "rouge1": 0.0,
            "rouge2": 0.0,
            "rougeL": 0.0,
        }


# Data collator
collator = DataCollatorForSeq2Seq(
    tokenizer, model=model, padding=True
)


# Training arguments
args = Seq2SeqTrainingArguments(
    output_dir="/content/drive/MyDrive/model_checkpoints",
    save_strategy="steps",
    save_steps=500,
    save_total_limit=3,
    eval_strategy="steps",
    eval_steps=500,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=1,
    eval_accumulation_steps=16,
    predict_with_generate=True,
    generation_num_beams=2,
    generation_max_length=256,
    num_train_epochs=2,
    learning_rate=3e-5,
    warmup_steps=500,
    weight_decay=0.01,
    fp16=True,
    logging_strategy="steps",
    logging_steps=100,
    load_best_model_at_end=True,
    metric_for_best_model="rouge1",
    greater_is_better=True,
    report_to=["tensorboard"],
    logging_dir="logs",
)

# Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
    data_collator=collator,
    compute_metrics=compute_metrics,
)

# Resuming from checkpoint (in my case, 3600)
resume_from = "/content/checkpoint-NUMBER"

# Check if it exists
if os.path.exists(resume_from):
    print(f"Using checkpoint: {resume_from}")
    trainer.train(resume_from_checkpoint=resume_from)
else:
    print("Checkpoint not found, starting fresh")
    trainer.train()


# Save the model
trainer.save_model("final_model")
tokenizer.save_pretrained("final_model")

Retrieving folder contents


Processing file 1G4UfMB1ecY-OqVcHMfUMAih1B_ZFAf4x config.json
Processing file 1OeMWsh5cmQNdFVQU0oZL-Bu3BxXILPdn generation_config.json
Processing file 1A52l4M0oI2DOuLkArVPFy56YalQZbWnx merges.txt
Processing file 1w8HWz08pf7ImIWTpfCndh23xz1t8ZkzL model.safetensors
Processing file 1uOL3kUNc_MbFPWdomNK4p9mkUlNagwwR rng_state.pth
Processing file 1qMICI-uzpYoiy2G29feXgTs7U0o6H_Pe scaler.pt
Processing file 1tZ62BGy4ze2kEDrG7qfN30r89Op550a3 scheduler.pt
Processing file 1l0PXvn7WI0YMRjB4Wmh7NMDJ1DdT5vMH special_tokens_map.json
Processing file 1cZ9w8CNKeHDh1IvGN4OmaDDn28JXE2_f tokenizer_config.json
Processing file 1sskh5Ae6ZIKGJIMLhTAMiMRlTZpa32jy tokenizer.json
Processing file 1cANOyGF2cEyBu1-8OFTp8Dr_GBBV9wWZ trainer_state.json
Processing file 1iZ6emNFgnnGJzX_lLUKLqDZLhMwLbUIw training_args.bin
Processing file 1DzLvo0rIu_zomtcev6sfXSq0dC8FitsX vocab.json


Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1G4UfMB1ecY-OqVcHMfUMAih1B_ZFAf4x
To: /content/checkpoint-3600/config.json
100%|██████████| 1.59k/1.59k [00:00<00:00, 6.03MB/s]
Downloading...
From: https://drive.google.com/uc?id=1OeMWsh5cmQNdFVQU0oZL-Bu3BxXILPdn
To: /content/checkpoint-3600/generation_config.json
100%|██████████| 328/328 [00:00<00:00, 1.38MB/s]
Downloading...
From: https://drive.google.com/uc?id=1A52l4M0oI2DOuLkArVPFy56YalQZbWnx
To: /content/checkpoint-3600/merges.txt
100%|██████████| 456k/456k [00:00<00:00, 6.34MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1w8HWz08pf7ImIWTpfCndh23xz1t8ZkzL
From (redirected): https://drive.google.com/uc?id=1w8HWz08pf7ImIWTpfCndh23xz1t8ZkzL&confirm=t&uuid=478ca686-a8e7-47c7-9237-4888cb8565a2
To: /content/checkpoint-3600/model.safetensors
100%|██████████| 1.63G/1.63G [00:22<00:00, 71.2MB/s]
Downloading...
Fro

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/tmp/ipython-input-4252444020.py:148: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


...

Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel
3800,0.504300,1.063339,0.453779,0.216304,0.302864
4000,0.495500,1.089444,0.422003,0.190148,0.273031
4200,0.776700,1.019952,0.432371,0.200777,0.278550
4400,0.791000,1.005972,0.435188,0.202099,0.292370
4600,0.811200,1.013479,0.436497,0.208488,0.284821


Predictions shape: (100, 176)
Predictions dtype: int64
Predictions min/max: -100/50141
After processing - Predictions min/max: -100/50141
Tokenizer vocab size: 50265
Clipping to max_token_id: 50264
Successfully decoded 100 predictions
Successfully decoded 100 labels
ROUGE scores computed successfully


/usr/local/lib/python3.12/dist-packages/transformers/modeling_utils.py:3917: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


Predictions shape: (100, 256)
Predictions dtype: int64
Predictions min/max: -100/50141
After processing - Predictions min/max: -100/50141
Tokenizer vocab size: 50265
Clipping to max_token_id: 50264
Successfully decoded 100 predictions
Successfully decoded 100 labels
ROUGE scores computed successfully
Predictions shape: (100, 176)
Predictions dtype: int64
Predictions min/max: -100/50141
After processing - Predictions min/max: -100/50141
Tokenizer vocab size: 50265
Clipping to max_token_id: 50264
Successfully decoded 100 predictions
Successfully decoded 100 labels
ROUGE scores computed successfully
Predictions shape: (100, 161)
Predictions dtype: int64
Predictions min/max: -100/50118
After processing - Predictions min/max: -100/50118
Tokenizer vocab size: 50265
Clipping to max_token_id: 50264
Successfully decoded 100 predictions
Successfully decoded 100 labels
ROUGE scores computed successfully
Predictions shape: (100, 166)
Predictions dtype: int64
Predictions min/max: -100/50141
After p

KeyboardInterrupt: 

In [39]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Load model and tokenizer from a specific checkpoint
checkpoint_path = "/content/your-checkpoint-path"

print("Loading model from checkpoint...")
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint_path)
tokenizer = AutoTokenizer.from_pretrained(checkpoint_path)

print("Model loaded successfully!")


Loading model from checkpoint...


/usr/local/lib/python3.12/dist-packages/transformers/models/bart/configuration_bart.py:177: UserWarning: Please make sure the config includes `forced_bos_token_id=0` in future versions. The config can simply be saved and uploaded again to be fixed.
  warnings.warn(


✅ Model loaded successfully!


In [41]:
# Test function for news summarization
def test_summarization(article_text, max_length=128):
    inputs = tokenizer(article_text, return_tensors="pt", truncation=True, max_length=512)

    with torch.no_grad():
        outputs = model.generate(
            inputs.input_ids,
            max_length=max_length,
            num_beams=4,
            early_stopping=True,
            do_sample=False,
            length_penalty=1.0
        )

    summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return summary

# Test 1: Technology News
test_article_1 = """
Apple announced today that it will be releasing a major software update for its iPhone lineup next month. The iOS 18.2 update includes several new artificial intelligence features, improved battery management, and enhanced security protocols. The company's CEO stated that this update represents the most significant advancement in iPhone software in the past three years. Early beta testers have reported improvements in app performance and faster charging speeds. The update will be available for iPhone 12 and newer models, with older devices receiving a limited version of the features. Apple's stock price rose 3% following the announcement, as investors showed confidence in the company's continued innovation in the mobile technology sector.
"""

print("=== TEST 1: Technology News ===")
print("Original article length:", len(test_article_1.split()))
print("\nGenerated summary:")
summary_1 = test_summarization(test_article_1)
print(summary_1)
print(f"\nSummary length: {len(summary_1.split())} words")

# Test 2: Politics/Economy News
test_article_2 = """
The Federal Reserve announced a 0.25% interest rate cut yesterday, marking the third reduction this year as officials attempt to stimulate economic growth amid concerns about global trade tensions. Fed Chair Jerome Powell explained that the decision was made to support continued expansion and maintain price stability. Economists had mixed reactions to the announcement, with some arguing that further cuts may be necessary while others warned about potential inflationary pressures. The stock market responded positively, with major indices gaining over 2% in after-hours trading. Small businesses are expected to benefit from lower borrowing costs, while savers may see reduced returns on deposits. The next Federal Open Market Committee meeting is scheduled for December, where additional rate changes will be considered based on economic data and employment figures.
"""

print("\n\n=== TEST 2: Politics/Economy News ===")
print("Original article length:", len(test_article_2.split()))
print("\nGenerated summary:")
summary_2 = test_summarization(test_article_2)
print(summary_2)
print(f"\nSummary length: {len(summary_2.split())} words")

def clean_summary(text):
    # Remove spaces before punctuation
    import re
    text = re.sub(r'\s+([.,!?;:])', r'\1', text)
    # Clean up multiple spaces
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

# Apply to your summary
clean_summary_2 = clean_summary(summary_2)
print("Cleaned:", clean_summary_2)


=== TEST 1: Technology News ===
Original article length: 111

Generated summary:
Apple will release iOS 18.2 next month .
The update will be available for iPhone 12 and newer models, with older devices receiving a limited version .
Apple's stock price rose 3% following the announcement .
Investors showed confidence in the company's continued innovation .

Summary length: 45 words


=== TEST 2: Politics/Economy News ===
Original article length: 127

Generated summary:
The Federal Reserve announced a 0.25% interest rate cut yesterday .
It is the third reduction this year .
The move is expected to stimulate economic growth .
Analysts had mixed reactions to the announcement .
Some argued that further cuts may be necessary while others warned about potential inflationary pressures .

Summary length: 52 words
Cleaned: The Federal Reserve announced a 0.25% interest rate cut yesterday. It is the third reduction this year. The move is expected to stimulate economic growth. Analysts had mixed re